# Device

In [ ]:
import torch                         #Включаем видеокарту если есть
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Используется устройство: {device}")

# Bayesian  optimization hyperparameter

In [ ]:
import pandas as pd
import optuna
from neuralprophet import NeuralProphet
from sklearn.metrics import mean_absolute_error
from joblib import Parallel, delayed

# ✅ Загружаем и очищаем данные
data = pd.read_csv('C:/Users/bondarenKovv/Desktop/Python/NeuralProphet/Magazin/Sales.csv', sep=';', low_memory=False)
data['Дата'] = pd.to_datetime(data['Дата'], dayfirst=True)
data = data.rename(columns={
    'Дата': 'ds',
    'Номер Магазина': 'store',
    '(Сутки).(Сумма продаж в фактических ценах реализации(валюта))': 'y'
})
data['y'] = data['y'].astype(str).str.replace(',', '.').astype(float)
data = data.drop_duplicates(subset=['store', 'ds']).dropna()
data = data.sort_values(by=['store', 'ds']).reset_index(drop=True)
data = data[data['ds'] >= '2023-01-01']

# 🔹 Группируем данные по магазинам
store_groups = data.groupby('store')

# 🔥 Функция для оптимизации гиперпараметров
def objective(trial, store_data):
    if len(store_data) < 60:
        return float('inf')
    
    params = {
        'n_changepoints': trial.suggest_int('n_changepoints', 5, 35),
        'changepoints_range': trial.suggest_float('changepoints_range', 0.8, 0.95),
        'yearly_seasonality': trial.suggest_int('yearly_seasonality', 1, 20),
        'weekly_seasonality': trial.suggest_int('weekly_seasonality', 1, 7),
        'seasonality_mode': trial.suggest_categorical('seasonality_mode', ['additive', 'multiplicative']),
        'trend_reg': trial.suggest_float('trend_reg', 0.1, 1.0),
        'seasonality_reg': trial.suggest_float('seasonality_reg', 0.01, 1.0),
    }
    
    train_size = int(len(store_data) * 0.8)
    train_data, test_data = store_data[:train_size], store_data[train_size:]
    if len(test_data) == 0:
        return float('inf')
    
    train_data = train_data[['ds', 'y']]
    test_data = test_data[['ds', 'y']]
    
    model = NeuralProphet(**params, drop_missing=True)
    model.fit(train_data, freq='D')
    
    future = model.make_future_dataframe(df=train_data, periods=len(test_data))
    forecast = model.predict(future)
    
    if forecast.empty or len(forecast) < len(test_data):
        return float('inf')
    
    mae = mean_absolute_error(test_data['y'].values, forecast['yhat1'].values[:len(test_data)])
    return mae

# 🔥 Оптимизация гиперпараметров для каждого магазина
def optimize_for_store(store_id, store_data):
    study = optuna.create_study(direction='minimize')
    study.optimize(lambda trial: objective(trial, store_data), n_trials=100)
    
    best_params = study.best_params
    best_params['store'] = store_id
    best_params['cv_score'] = study.best_value
    return best_params

# 🔥 Параллельная оптимизация
best_params_list = Parallel(n_jobs=-1, verbose=10)(
    delayed(optimize_for_store)(store_id, store_data) for store_id, store_data in store_groups
)

# 🔹 Сохранение параметров
best_params_df = pd.DataFrame(best_params_list)
best_params_df.to_excel('C:/Users/bondarenKovv/Desktop/Python/NeuralProphet/Magazin/best_params.xlsx', index=False)

print("✅ Гиперпараметры рассчитаны и сохранены!")

# Плюс CUDA

In [ ]:
import pandas as pd
import optuna
import torch
from neuralprophet import NeuralProphet
from sklearn.metrics import mean_absolute_error
from joblib import Parallel, delayed

# 🔥 Принудительно используем GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_default_tensor_type('torch.cuda.FloatTensor' if torch.cuda.is_available() else 'torch.FloatTensor')
print(f"🔍 Используется устройство: {device}")

# ✅ Загружаем данные
data = pd.read_csv('C:/Users/bondarenKovv/Desktop/Python/NeuralProphet/Magazin/Sales.csv', sep=';', low_memory=False)
data['Дата'] = pd.to_datetime(data['Дата'], dayfirst=True)
data = data.rename(columns={'Дата': 'ds', 'Номер Магазина': 'store', '(Сутки).(Сумма продаж в фактических ценах реализации(валюта))': 'y'})
data['y'] = data['y'].astype(str).str.replace(',', '.').astype(float)
data = data.drop_duplicates(subset=['store', 'ds']).dropna()
data = data.sort_values(by=['store', 'ds']).reset_index(drop=True)
data = data[data['ds'] >= '2023-01-01']

# 🔹 Группируем данные по магазинам
store_groups = data.groupby('store')

# 🔥 Функция оптимизации гиперпараметров
def objective(trial, store_data):
    if len(store_data) < 60:
        return float('inf')

    params = {
        'n_changepoints': trial.suggest_int('n_changepoints', 5, 35),
        'changepoints_range': trial.suggest_float('changepoints_range', 0.8, 0.95),
        'yearly_seasonality': trial.suggest_int('yearly_seasonality', 1, 20),
        'weekly_seasonality': trial.suggest_int('weekly_seasonality', 1, 7),
        'seasonality_mode': trial.suggest_categorical('seasonality_mode', ['additive', 'multiplicative']),
        'trend_reg': trial.suggest_float('trend_reg', 0.1, 1.0),
        'seasonality_reg': trial.suggest_float('seasonality_reg', 0.01, 1.0),
        'learning_rate': 0.01,  # ✅ Добавлено для ускорения
        'batch_size': 256,  # ✅ Ускорение обучения
    }

    train_size = int(len(store_data) * 0.8)
    train_data, test_data = store_data[:train_size], store_data[train_size:]
    if len(test_data) == 0:
        return float('inf')

    train_data = train_data[['ds', 'y']]
    test_data = test_data[['ds', 'y']]

    # ✅ Создаем модель (убрали device, но используем GPU внутри PyTorch)
    model = NeuralProphet(**params)
    model.fit(train_data, freq='D')

    # ✅ Предсказание
    future = model.make_future_dataframe(df=train_data, periods=len(test_data))
    forecast = model.predict(future)

    if forecast.empty or len(forecast) < len(test_data):
        return float('inf')

    mae = mean_absolute_error(test_data['y'].values, forecast['yhat1'].values[:len(test_data)])
    return mae

# 🔥 Функция для запуска оптимизации
def optimize_for_store(store_id, store_data):
    print(f"🚀 Оптимизация для магазина {store_id} началась...")

    study = optuna.create_study(direction='minimize')
    study.optimize(lambda trial: objective(trial, store_data), n_trials=100)  # ⚡ Увеличено число итераций

    best_params = study.best_params
    best_params['store'] = store_id
    best_params['cv_score'] = study.best_value

    print(f"✅ Оптимизация завершена для магазина {store_id}!")
    return best_params

# 🔥 Параллельная оптимизация с `loky`
best_params_list = Parallel(n_jobs=-1, verbose=10, backend="loky")(
    delayed(optimize_for_store)(store_id, store_data) for store_id, store_data in store_groups
)

# 🔹 Сохранение параметров
best_params_df = pd.DataFrame(best_params_list)
best_params_df.to_excel('C:/Users/bondarenKovv/Desktop/Python/NeuralProphet/Magazin/best_params.xlsx', index=False)

print("✅ Гиперпараметры рассчитаны и сохранены!")


# Гиперы Плюс Куда плюс кросс валидация

In [ ]:
import pandas as pd
import optuna
import torch
from neuralprophet import NeuralProphet
from sklearn.metrics import mean_absolute_error
from joblib import Parallel, delayed
from sklearn.model_selection import TimeSeriesSplit

# 🔥 Принудительно используем GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_default_tensor_type('torch.cuda.FloatTensor' if torch.cuda.is_available() else 'torch.FloatTensor')
print(f"🔍 Используется устройство: {device}")

# ✅ Загружаем данные
data = pd.read_csv('C:/Users/bondarenKovv/Desktop/Python/NeuralProphet/Magazin/Sales.csv', sep=';', low_memory=False)
data['Дата'] = pd.to_datetime(data['Дата'], dayfirst=True)
data = data.rename(columns={'Дата': 'ds', 'Номер Магазина': 'store', '(Сутки).(Сумма продаж в фактических ценах реализации(валюта))': 'y'})
data['y'] = data['y'].astype(str).str.replace(',', '.').astype(float)
data = data.drop_duplicates(subset=['store', 'ds']).dropna()
data = data.sort_values(by=['store', 'ds']).reset_index(drop=True)
data = data[data['ds'] >= '2023-01-01']

# 🔹 Группируем данные по магазинам
store_groups = data.groupby('store')

# 🔥 Функция оптимизации гиперпараметров с кросс-валидацией
def objective(trial, store_data):
    if len(store_data) < 60:
        return float('inf')

    params = {
        'n_changepoints': trial.suggest_int('n_changepoints', 5, 35),
        'changepoints_range': trial.suggest_float('changepoints_range', 0.8, 0.95),
        'yearly_seasonality': trial.suggest_int('yearly_seasonality', 1, 20),
        'weekly_seasonality': trial.suggest_int('weekly_seasonality', 1, 7),
        'seasonality_mode': trial.suggest_categorical('seasonality_mode', ['additive', 'multiplicative']),
        'trend_reg': trial.suggest_float('trend_reg', 0.1, 1.0),
        'seasonality_reg': trial.suggest_float('seasonality_reg', 0.01, 1.0),
        'learning_rate': 0.01,  # ✅ Добавлено для ускорения
        'batch_size': 256,  # ✅ Ускорение обучения
    }

    tscv = TimeSeriesSplit(n_splits=3)  # ✅ 3 фолда
    scores = []

    for train_idx, test_idx in tscv.split(store_data):
        train_data, test_data = store_data.iloc[train_idx], store_data.iloc[test_idx]
        
        if len(test_data) == 0:
            continue

        train_data = train_data[['ds', 'y']]
        test_data = test_data[['ds', 'y']]

        # ✅ Создаем новую модель на каждой итерации
        model = NeuralProphet(**params)
        model.fit(train_data, freq='D')

        # ✅ Предсказание
        future = model.make_future_dataframe(df=train_data, periods=len(test_data))
        forecast = model.predict(future)

        if forecast.empty or len(forecast) < len(test_data):
            continue

        mae = mean_absolute_error(test_data['y'].values, forecast['yhat1'].values[:len(test_data)])
        scores.append(mae)

    return sum(scores) / len(scores) if scores else float('inf')

# 🔥 Функция для запуска оптимизации
def optimize_for_store(store_id, store_data):
    print(f"🚀 Оптимизация для магазина {store_id} началась...")

    study = optuna.create_study(direction='minimize')
    study.optimize(lambda trial: objective(trial, store_data), n_trials=100)  # ⚡ 100 итераций

    best_params = study.best_params
    best_params['store'] = store_id
    best_params['cv_score'] = study.best_value

    print(f"✅ Оптимизация завершена для магазина {store_id}!")
    return best_params

# 🔥 Параллельная оптимизация с `loky`
best_params_list = Parallel(n_jobs=-1, verbose=10, backend="loky")(
    delayed(optimize_for_store)(store_id, store_data) for store_id, store_data in store_groups
)

# 🔹 Сохранение параметров
best_params_df = pd.DataFrame(best_params_list)
best_params_df.to_excel('C:/Users/bondarenKovv/Desktop/Python/NeuralProphet/Magazin/best_params.xlsx', index=False)

print("✅ Гиперпараметры рассчитаны и сохранены!")


WARNING - (py.warnings._showwarnmsg) - C:\Users\bondarenkovv\AppData\Local\anaconda3\Lib\site-packages\torch\__init__.py:1144: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\tensor\python_tensor.cpp:434.)
  _C._set_default_tensor_type(t)



🔍 Используется устройство: cuda


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed: 31.1min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed: 62.8min
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed: 63.0min


# Grid

In [ ]:
# 52488 комбинаций перебора.для 1 магазина при условии паралельных вычислений 16 комбинаций в минуту 
# на процесооре будет считать 54 часа...
param_grid = {
    'n_changepoints': [5, 10, 20],  # Количество точек изменения тренда
    'changepoints_range': [0.8, 0.9, 1.0],  # Диапазон для выбора точек изменения
    'yearly_seasonality': [True, 5, 10],  # True - авто, 5 или 10 для настройки
    'weekly_seasonality': [True, 3, 5],  # True - авто, или точные значения
    'daily_seasonality': [False, 10, 20],  # Добавляем дневную сезонность, если она есть
    'seasonality_mode': ['additive', 'multiplicative'],  # Режим сезонности
    'growth': ['linear', 'discontinuous'],  # Режим роста
    'n_lags': [0, 5, 10],  # Задержки для добавления автокорреляции
    'ar_regularization': [0, 0.1, 0.5],  # Регуляризация для автокорреляции
    'trend_reg': [0, 0.1, 0.5],  # Регуляризация для тренда
    'seasonality_reg': [0, 0.1, 0.5],  # Регуляризация для сезонности
}

# Рассчет прогноза с гиперпараметрами Bayesian 

In [ ]:
import warnings
import pandas as pd
from neuralprophet import NeuralProphet
from joblib import Parallel, delayed

warnings.filterwarnings("ignore")

# Блок 1: Загрузка данных и предварительная обработка
data = pd.read_csv('C:/Users/bondarenKovv/Desktop/Python/NeuralProphet/Magazin/Sales.csv', sep=';', low_memory=False)
data['Дата'] = pd.to_datetime(data['Дата'], dayfirst=True)
data = data[(data['Дата'] >= '2023-01-01')]

data = data.rename(columns={'Дата': 'ds', 'Номер Магазина': 'store', '(Сутки).(Сумма продаж в фактических ценах реализации(валюта))': 'y'})
data['y'] = data['y'].str.replace(',', '.').astype(float)
data.dropna(inplace=True)

# Блок 2: Загрузка гиперпараметров модели
try:
    model_params = pd.read_excel('best_model_params_cv1.xlsx')
except FileNotFoundError:
    model_params = pd.DataFrame(columns=['store', 'n_changepoints', 'changepoints_range', 'yearly_seasonality', 
                                         'weekly_seasonality', 'seasonality_mode', 'trend_reg', 'seasonality_reg', 'monthly_seasonality'])

# Блок 3: Предпраздничные даты и исключенные магазины
pre_holiday_dates = ['2025-02-22', '2025-03-07', '2025-05-08', '2025-06-11', '2025-04-20', '2025-05-02', '2025-05-03',
                     '2024-12-28', '2024-12-29', '2024-12-30', '2024-12-31', '2025-12-28', '2025-12-29', '2025-12-30', '2025-12-31']
excluded_stores = [149, 155, 164, 111, 123, 136, 127, 151, 109]
standard_stores = [187, 188]

# Функция для прогнозирования для одного магазина
def forecast_store(store):
    if store in excluded_stores:
        return pd.DataFrame()

    store_data = data[data['store'] == store]
    if len(store_data) < 2:
        print(f'Пропускаем магазин {store} из-за недостатка данных')
        return pd.DataFrame()

    store_data = store_data.drop_duplicates(subset=['ds'])
    store_data = store_data.drop(columns=['store'])

    # Проверка, используется ли магазин в стандартных гиперпараметрах
    if store in standard_stores:
        model = NeuralProphet()
    else:
        params = model_params[model_params['store'] == store]
        if params.empty:
            print(f'Не найдены гиперпараметры для магазина {store}, пропускаем его')
            return pd.DataFrame()

        model = NeuralProphet(
            n_changepoints=int(params['n_changepoints'].values[0]),
            changepoints_range=params['changepoints_range'].values[0],
            yearly_seasonality=params['yearly_seasonality'].values[0],
            weekly_seasonality=params['weekly_seasonality'].values[0],
            seasonality_mode=params['seasonality_mode'].values[0],
            trend_reg=params['trend_reg'].values[0],
            seasonality_reg=params['seasonality_reg'].values[0]
        )

        # Добавляем месячную сезонность
        if 'monthly_seasonality' in params.columns:
            model.add_seasonality(name='monthly', period=30.4, fourier_order=int(params['monthly_seasonality'].values[0]))

    model.fit(store_data, freq='D')
    future = model.make_future_dataframe(store_data, periods=50)
    forecast = model.predict(future)
    forecast['store'] = store

    for pre_holiday_date in pre_holiday_dates:
        mask = forecast['ds'] == pd.to_datetime(pre_holiday_date)
        forecast.loc[mask, 'yhat1'] *= 1.165

    return forecast

# Получение уникальных ID магазинов
store_ids = data['store'].unique()

# Параллельное прогнозирование для всех магазинов
all_forecasts = Parallel(n_jobs=-1, verbose=10)(delayed(forecast_store)(store) for store in store_ids)

# Объединение всех прогнозов в один DataFrame
all_forecasts = pd.concat(all_forecasts, ignore_index=True)

# Сохранение результатов прогноза в новом Excel-файле
selected_columns = ['ds', 'store', 'yhat1']
all_forecasts[selected_columns].to_excel('Февральbest_model_params_cv1.xlsx', index=False)


# Графики

In [ ]:
import matplotlib.pyplot as plt                     # График по всем магазинам
import matplotlib.dates as mdates
import pandas as pd

# Загрузка данных прогнозов и фактических продаж
data = pd.read_csv('C:/Users/bondarenKovv/Desktop/Python/NeuralProphet/Magazin/Sales.csv', sep=';', low_memory=False)
data['Дата'] = pd.to_datetime(data['Дата'], dayfirst=True)

# Переименование столбцов для удобства
data = data.rename(columns={
    'Дата': 'ds', 
    'Номер Магазина': 'store', 
    '(Сутки).(Сумма продаж в фактических ценах реализации(валюта))': 'y'
})

# Преобразование столбца 'y' в числовой формат
data['y'] = data['y'].str.replace(',', '.').astype(float)

# Фильтрация данных начиная с 2024 года
data = data[data['ds'] >= '2023-12-01']

# Суммирование фактических данных по всем магазинам
total_sales = data.groupby('ds').agg({'y': 'sum'}).reset_index()

# Загрузка прогнозов
all_forecasts = pd.read_excel('Forecast_NeuralProphet_with_reg.xlsx',sheet_name='Sheet1', index_col=None)  # Измени имя файла, если нужно
all_forecasts['ds'] = pd.to_datetime(all_forecasts['ds'])
filtered_forecasts = all_forecasts[all_forecasts['ds'] >= '2023-12-01']

# Суммирование прогнозов по всем магазинам
summed_forecasts = filtered_forecasts.groupby('ds').agg({'yhat1': 'sum'}).reset_index()  # Измени 'yhat' на 'yhat1'

# Создание общего графика суммарного прогноза с фактическими данными
plt.figure(figsize=(14, 8))

# Линия фактических продаж
plt.plot(total_sales['ds'], total_sales['y'], label='Продажи', color='red', linewidth=2)

# Линия суммарного прогноза
plt.plot(summed_forecasts['ds'], summed_forecasts['yhat1'], label='Прогноз', color='blue', linestyle='--', linewidth=2)  # Измени 'yhat' на 'yhat1'

# Настройка заголовка и меток осей
plt.title('Динамика Продаж и Прогноза', fontsize=16)
plt.xlabel('Понедельники', fontsize=14)
plt.ylabel('Продажи в руб.', fontsize=14)

# Отображение легенды
plt.legend(fontsize=12)

# Отображение сетки
plt.grid(True, which='both', linestyle='--', linewidth=0.5)

# Улучшение форматирования осей
plt.xticks(rotation=45)
plt.tight_layout()

# Добавление разделителей тысяч для оси Y
plt.gca().get_yaxis().set_major_formatter(plt.matplotlib.ticker.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))

# Добавление меток начала каждой недели на ось X
ax = plt.gca()
ax.xaxis.set_major_locator(mdates.WeekdayLocator(byweekday=mdates.MO))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))

# Показ графика
plt.show()


In [ ]:
import matplotlib.pyplot as plt                 # График по выбранному магазину
import matplotlib.dates as mdates
import pandas as pd

# Загрузка данных прогнозов и фактических продаж
data = pd.read_csv('C:/Users/bondarenKovv/Desktop/Python/NeuralProphet/Magazin/Sales.csv', sep=';', low_memory=False)
data['Дата'] = pd.to_datetime(data['Дата'], dayfirst=True)

# Переименование столбцов для удобства
data = data.rename(columns={
    'Дата': 'ds', 
    'Номер Магазина': 'store', 
    '(Сутки).(Сумма продаж в фактических ценах реализации(валюта))': 'y'
})

# Преобразование столбца 'y' в числовой формат
data['y'] = data['y'].str.replace(',', '.').astype(float)

# Фильтрация данных начиная с 2024 года
data = data[data['ds'] >= '2023-12-01']

# Фильтрация данных по выбранному магазину (например, магазин с ID 148)
store_id = 152
store_data = data[data['store'] == store_id]

# Загрузка прогнозов
all_forecasts = pd.read_excel('Forecast_NeuralProphet_with_reg.xlsx', index_col=None)

# Вывод доступных столбцов для диагностики
print(all_forecasts.columns)  # Добавлено для отладки

# Проверка названия столбца 'ds'
if 'ds' in all_forecasts.columns:
    all_forecasts['ds'] = pd.to_datetime(all_forecasts['ds'])
else:
    all_forecasts[' Дата'] = pd.to_datetime(all_forecasts[' Дата'])  # Измени имя, если нужно

filtered_forecasts = all_forecasts[all_forecasts['ds'] >= '2023-12-01']  # Измените имя столбца при необходимости

# Фильтрация прогнозов по выбранному магазину
store_forecasts = filtered_forecasts[filtered_forecasts['store'] == store_id]

# Создание графика прогноза с фактическими данными для выбранного магазина
plt.figure(figsize=(14, 8))

# Линия фактических продаж
plt.plot(store_data['ds'], store_data['y'], label='Продажи', color='red', linewidth=2)

# Линия прогноза
plt.plot(store_forecasts['ds'], store_forecasts['yhat1'], label='Прогноз', color='blue', linestyle='--', linewidth=2)

# Настройка заголовка и меток осей
plt.title(f'Динамика Продаж и Прогноза для магазина {store_id}', fontsize=16)
plt.xlabel('Понедельники', fontsize=14)
plt.ylabel('Продажи в руб.', fontsize=14)

# Отображение легенды
plt.legend(fontsize=12)

# Отображение сетки
plt.grid(True, which='both', linestyle='--', linewidth=0.5)

# Улучшение форматирования осей
plt.xticks(rotation=45)
plt.tight_layout()

# Добавление разделителей тысяч для оси Y
plt.gca().get_yaxis().set_major_formatter(plt.matplotlib.ticker.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))

# Добавление меток начала каждой недели на ось X
ax = plt.gca()
ax.xaxis.set_major_locator(mdates.WeekdayLocator(byweekday=mdates.MO))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))

# Показ графика
plt.show()
